In [1]:
import pandas as pd


faculty_info = pd.read_csv('../data_clean/faculty_info_clean.csv')
course_survey = pd.read_csv('../data_clean/course_survey_clean.csv')
student_comments = pd.read_csv('../data_clean/student_comments_clean.csv')


: 

In [ ]:
course_survey['rating_score'] = (
    course_survey['rating_score']
    .astype(str)
    .str.strip()
    .str.replace(',', '.')
    .replace({'': None, 'nan': None, 'NaN': None})
    .astype(float)
)

print("rating_score sau khi làm sạch (10 dòng đầu):")
print(course_survey['rating_score'].head(10))
print("\nKiểu dữ liệu:", course_survey['rating_score'].dtype)
print("Số giá trị bị thiếu (NaN):", course_survey['rating_score'].isna().sum())

In [ ]:
df_khao_sat = course_survey.merge(faculty_info, on='faculty_id', how='left')

print("Sau khi merge thông tin giảng viên:")
print(f"Kích thước dataframe: {df_khao_sat.shape[0]} dòng, {df_khao_sat.shape[1]} cột")
print("\n5 dòng đầu:")
print(df_khao_sat[['survey_id', 'faculty_id', 'full_name', 'department', 'rating_score']].head())

In [ ]:
comments_agg = (
    student_comments.groupby('survey_id')
    .agg({
        'comment_text': lambda x: ' | '.join(x.dropna().astype(str)),  # Gộp comment
        'sentiment_label': 'first'  # Lấy nhãn cảm xúc đầu tiên (hoặc có thể dùng mode)
    })
    .reset_index()
)

print("Số survey_id có bình luận sau khi gộp:", comments_agg.shape[0])

# Merge vào dataframe chính
df = df_khao_sat.merge(comments_agg, on='survey_id', how='left')

# Điền giá trị mặc định cho các khảo sát không có bình luận
df['sentiment_label'] = df['sentiment_label'].fillna('Trung lập')

print("\nSau khi merge bình luận:")
print(f"Kích thước cuối cùng: {df.shape[0]} dòng, {df.shape[1]} cột")
print("\n5 dòng đầu (đầy đủ thông tin):")
print(df.head())

In [ ]:
diem_tb_giangvien = (
    df.groupby(['faculty_id', 'full_name', 'department'], as_index=False)
    .agg(
        diem_trung_binh=('rating_score', 'mean'),
        so_khao_sat=('rating_score', 'count')
    )
    .round({'diem_trung_binh': 2})
    .sort_values('diem_trung_binh', ascending=False)
    .reset_index(drop=True)
)

print("1. Điểm trung bình theo giảng viên (top 20):")
print(diem_tb_giangvien.head(20))

#2.Điểm trung bình theo khoa/bộ môn
diem_tb_khoa = (
    df.groupby('department')['rating_score']
    .mean()
    .round(2)
    .sort_values(ascending=False)
    .reset_index(name='diem_trung_binh')
)

print("\n2. Điểm trung bình theo khoa/bộ môn:")
print(diem_tb_khoa)

In [ ]:
bang_cheo = (
    df.groupby(['department', 'student_level'])['rating_score']
    .mean()
    .round(2)
    .unstack(fill_value=0)
)

print("3. Điểm trung bình theo khoa và mức độ sinh viên:")
print(bang_cheo)

#4.Tỷ lệ cảm xúc theo khoa
ty_le_cam_xuc = df.groupby(['department', 'sentiment_label']).size().unstack(fill_value=0)

for label in ['Tích cực', 'Tiêu cực', 'Trung lập']:
    if label not in ty_le_cam_xuc.columns:
        ty_le_cam_xuc[label] = 0

ty_le_cam_xuc['Tong'] = ty_le_cam_xuc.sum(axis=1)
ty_le_cam_xuc['Tich_cuc_%'] = (ty_le_cam_xuc['Tích cực'] / ty_le_cam_xuc['Tong'] * 100).round(1)
ty_le_cam_xuc['Tieu_cuc_%'] = (ty_le_cam_xuc['Tiêu cực'] / ty_le_cam_xuc['Tong'] * 100).round(1)
ty_le_cam_xuc['Trung_lap_%'] = (ty_le_cam_xuc['Trung lập'] / ty_le_cam_xuc['Tong'] * 100).round(1)

print("\n4. Tỷ lệ cảm xúc theo khoa:")
print(ty_le_cam_xuc[['Tich_cuc_%', 'Tieu_cuc_%', 'Trung_lap_%']].sort_values('Tich_cuc_%', ascending=False))

In [ ]:
nguong = 5  #Chỉ lấy giảng viên có ít nhất 5 khảo sát

top_giangvien = diem_tb_giangvien[diem_tb_giangvien['so_khao_sat'] >= nguong].head(10)
thap_giangvien = diem_tb_giangvien[diem_tb_giangvien['so_khao_sat'] >= nguong].sort_values('diem_trung_binh').head(10)

print(f"Top 10 giảng viên ĐIỂM CAO NHẤT (≥ {nguong} khảo sát):")
print(top_giangvien[['full_name', 'department', 'diem_trung_binh', 'so_khao_sat']])

print(f"\nTop 10 giảng viên ĐIỂM THẤP NHẤT (≥ {nguong} khảo sát):")
print(thap_giangvien[['full_name', 'department', 'diem_trung_binh', 'so_khao_sat']])

#Lưu kết quả
diem_tb_giangvien.to_csv('diem_trung_binh_giang_vien.csv', index=False, encoding='utf-8-sig')
diem_tb_khoa.to_csv('diem_trung_binh_khoa.csv', index=False, encoding='utf-8-sig')
bang_cheo.to_csv('bang_cheo_khoa_nam.csv', encoding='utf-8-sig')
ty_le_cam_xuc[['Tich_cuc_%', 'Tieu_cuc_%', 'Trung_lap_%']].to_csv('ty_le_cam_xuc.csv', encoding='utf-8-sig')

print("\nĐã lưu tất cả kết quả ra file CSV!")